In [2]:
import pybullet as p
import time
import pybullet_data

sim_freq = 300
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())  # optionally
p.setGravity(0, 0, -9.81)
p.setTimeStep(1.0/sim_freq)
p.setRealTimeSimulation(0)
planeId = p.loadURDF("plane.urdf")
startPos = [0, 0, 0.5]
startOrientation = p.getQuaternionFromEuler([0, 0, 0])
robot = p.loadURDF("urdf/quadrobot.urdf", startPos, startOrientation, useFixedBase=1,
                        flags=p.URDF_USE_INERTIA_FROM_FILE)#+p.URDF_USE_SELF_COLLISION+p.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS)

In [1]:
import pybullet as p
jointIds = []
for j in range(p.getNumJoints(robot)):
    p.changeDynamics(robot, j, linearDamping=0, angularDamping=0)
    info = p.getJointInfo(robot, j)
    # print(info)
    #jointName = info[1]
    jointType = info[2]
    if (jointType == p.JOINT_REVOLUTE):
        jointIds.append(j)

joint_dir = [-1]*16
# joint_offset = [0, -1.57, 0, 0, 1.57, 0, 0, 1.57, 0, 0, -1.57, 0]
joint_offset = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

pybullet build time: Dec  1 2021 18:33:43


NameError: name 'robot' is not defined

In [2]:
from math import *

bx = 0.257
by = 0.093
bz = 0.0
L1 = 0.0905
L2 = 0.202
L3 = 0.17629

def fkine_R1(theta):
    ef_pos = [0]*3
    ef_pos[0] = bx + L3*cos(theta[1]+theta[2]) + L2*cos(theta[1])
    ef_pos[1] = -by - L3*sin(theta[0])*sin(theta[1]+theta[2]) - L1*cos(theta[0]) - L2*sin(theta[0])*sin(theta[1])
    ef_pos[2] = L3*cos(theta[0])*sin(theta[1]+theta[2]) - L1*sin(theta[0]) + L2*cos(theta[0])*sin(theta[1])
    return ef_pos

def fkine_R2(theta):
    ef_pos = [0]*3
    ef_pos[0] = -(bx + L3*cos(theta[1]+theta[2]) + L2*cos(theta[1]))
    ef_pos[1] = -by - L3*sin(theta[0])*sin(theta[1]+theta[2]) - L1*cos(theta[0]) - L2*sin(theta[0])*sin(theta[1])
    ef_pos[2] = -(L3*cos(theta[0])*sin(theta[1]+theta[2]) - L1*sin(theta[0]) + L2*cos(theta[0])*sin(theta[1]))
    return ef_pos

def fkine_L1(theta):
    ef_pos = [0]*3
    ef_pos[0] = bx + L3*cos(theta[1]+theta[2]) + L2*cos(theta[1])
    ef_pos[1] = -(-by - L3*sin(theta[0])*sin(theta[1]+theta[2]) - L1*cos(theta[0]) - L2*sin(theta[0])*sin(theta[1]))
    ef_pos[2] = -(L3*cos(theta[0])*sin(theta[1]+theta[2]) - L1*sin(theta[0]) + L2*cos(theta[0])*sin(theta[1]))
    return ef_pos

def fkine_L2(theta):
    ef_pos = [0]*3
    ef_pos[0] = -(bx + L3*cos(theta[1]+theta[2]) + L2*cos(theta[1]))
    ef_pos[1] = -(-by - L3*sin(theta[0])*sin(theta[1]+theta[2]) - L1*cos(theta[0]) - L2*sin(theta[0])*sin(theta[1]))
    ef_pos[2] = L3*cos(theta[0])*sin(theta[1]+theta[2]) - L1*sin(theta[0]) + L2*cos(theta[0])*sin(theta[1])
    return ef_pos

def fkine(theta):
    ef_pos = [0]*12
    ef_pos[0:3] = fkine_R1(theta[0:3])
    ef_pos[3:6] = fkine_L1(theta[3:6])
    ef_pos[6:9] = fkine_R2(theta[6:9])
    ef_pos[9:] = fkine_L2(theta[9:])
    return ef_pos

In [8]:
from math import *

bx = 0.257
by = 0.093
bz = 0.0
L1 = 0.0905
L2 = 0.202
L3 = 0.17629

def ikine_R1(p_ref, config="x"):
    if config == "m":
        sign1 = 1
    elif config == "o":
        sign1 = -1
    elif config == "x":
        sign1 = 1

    px = p_ref[2]
    py = -by - p_ref[1]
    pz = p_ref[0] - bx

    a2 = px**2 + py**2 - L1**2
    if a2 < 0:
        a2 = 0
    
    if py > 0:
        theta1 = (atan2(py, px) - atan2(L1, -sqrt(a2)))
    else:
        theta1 = (atan2(py, px)+2*pi - atan2(L1, -sqrt(a2)))
    
    a3 = (px**2 + py**2 + pz**2 - L1**2 - L2**2 - L3**2)/(2*L2*L3)
    if a3 > 1:
        theta3 = sign1*acos(1)
    else: 
        if a3 < -1:
            theta3 = sign1*acos(-1)
        else:
            theta3 = sign1*acos(a3)

    a1 = (L3*sin(theta3))**2 + (L3*cos(theta3) + L2)**2 - pz**2
    if a1 < 0 :
        a1 = 0
    theta2 = (atan2(L3*cos(theta3)+L2, L3*sin(theta3)) - atan2(pz, -sqrt(a1))) % -pi

    
    return [theta1, theta2, theta3]

def ikine_R2(p_ref):
    px = p_ref[2]
    py = -by - p_ref[1]
    pz = p_ref[0] + bx

    a2 = px**2 + py**2 - L1**2
    if a2 < 0:
        a2 = 0
    
    if py > 0:
        theta1 = (atan2(py, px) - atan2(L1, -sqrt(a2)))
    else:
        theta1 = (atan2(py, px)+2*pi - atan2(L1, -sqrt(a2)))
    
    a3 = (px**2 + py**2 + pz**2 - L1**2 - L2**2 - L3**2)/(2*L2*L3)
    if a3 > 1:
        theta3 = -acos(1)
    else: 
        if a3 < -1:
            theta3 = -acos(-1)
        else:
            theta3 = -acos(a3)

    a1 = (L3*sin(theta3))**2 + (L3*cos(theta3) + L2)**2 - pz**2
    if a1 < 0 :
        a1 = 0
    theta2 = (atan2(L3*cos(theta3)+L2, L3*sin(theta3)) - atan2(pz, sqrt(a1))) % -pi

    
    return [-theta1, -theta2, -theta3]

def ikine_L1(p_ref):
    px = p_ref[2]
    py = by - p_ref[1]
    pz = p_ref[0] - bx

    a2 = px**2 + py**2 - L1**2
    if a2 < 0:
        a2 = 0
    
    if py > 0:
        theta1 = (atan2(py, px) - atan2(L1, sqrt(a2))) - pi
    else:
        theta1 = (atan2(py, px)+2*pi - atan2(L1, sqrt(a2))) - pi
    
    a3 = (px**2 + py**2 + pz**2 - L1**2 - L2**2 - L3**2)/(2*L2*L3)
    if a3 > 1:
        theta3 = acos(1)
    else: 
        if a3 < -1:
            theta3 = acos(-1)
        else:
            theta3 = acos(a3)

    a1 = (L3*sin(theta3))**2 + (L3*cos(theta3) + L2)**2 - pz**2
    if a1 < 0 :
        a1 = 0
    theta2 = (atan2(L3*cos(theta3)+L2, L3*sin(theta3)) - atan2(pz, sqrt(a1))) % -pi

    
    return [theta1, theta2, theta3]

def ikine_L2(p_ref):
    px = p_ref[2]
    py = by - p_ref[1]
    pz = p_ref[0] + bx

    a2 = px**2 + py**2 - L1**2
    if a2 < 0:
        a2 = 0
    
    if py > 0:
        theta1 = -((atan2(py, px) - atan2(L1, sqrt(a2))) - pi)
    else:
        theta1 = -((atan2(py, px)+2*pi - atan2(L1, sqrt(a2))) - pi)
    
    a3 = (px**2 + py**2 + pz**2 - L1**2 - L2**2 - L3**2)/(2*L2*L3)
    if a3 > 1:
        theta3 = acos(1)
    else: 
        if a3 < -1:
            theta3 = acos(-1)
        else:
            theta3 = -acos(a3)

    a1 = (L3*sin(theta3))**2 + (L3*cos(theta3) + L2)**2 - pz**2
    if a1 < 0 :
        a1 = 0
    theta2 = (atan2(L3*cos(theta3)+L2, L3*sin(theta3)) - atan2(pz, sqrt(a1))) - pi

    
    return [theta1, theta2, theta3]
    

In [13]:
import math

lst = ikine_R2([-0.311, -0.2075, -0.0498])
lst_ikine = [degrees(x) for x in lst]
lst_real = [degrees(0.3501), degrees(-0.0508), degrees(2.6142)]
print(lst_real)
print(lst_ikine)
-(180-177.119)


[20.059252407530124, -2.910625599264582, 149.7826268030998]
[20.04144525951319, 177.1193531496005, 149.81834918761862]


-2.8810000000000002

In [ ]:
import random

A = [random.random()*0.20 for _ in range(3)]
phi = [random.random()*3.14 for _ in range(3)]
nu = [random.randint(1, 20) for _ in range(3)]

t = 0
ef_pos_pract = []
t_axis = []
ref_pos = [0]*12
it = 0

In [ ]:
#p_ref=[-2.909200617878336, 2.308450481393923, 1.6463290455433555]
p_offset = [0.63529, -0.1835, 0.0]
p_ref = [-0.257, 0.1835, -0.25]
p_base = [-0.257, 0.1835, -0.25]
p_ref_hist = [[0.257, 0.1835, 0.25]]
ef_pos_pract = []
theta_ref = []
for i in range (1000):
    
    p_ref[0] = 0.2*sin(5*t) + p_base[0]
    p_ref[1] = 0.15*sin(5*t + pi/2) + p_base[1]
    #p_ref[2] = 0.1*sin(5*t + pi/2) + p_base[2]
    p_ref_hist.append([p_ref[0], p_ref[1], p_ref[2]])
    
    # print(p_ref)
    ref_pos[9:12] = ikine_L2(p_ref)

    for i in range(len(jointIds)):
            p.setJointMotorControl2(robot, jointIds[i], p.POSITION_CONTROL,
                                    targetPosition=joint_dir[i]*ref_pos[i]-joint_offset[i],
                                    maxVelocity=15.0)#
    p.stepSimulation()
    t += 1.0/sim_freq
    
    ef_pos_pract_tmp1 = p.getLinkState(robot, 16)
    ef_pos_pract.append([x for x in ef_pos_pract_tmp1[0]])
    #theta_ref.append(ref_pos[9:12])
    t_axis.append(t)
    
    time.sleep(1.0/sim_freq)


In [ ]:
p_ref

[-0.418751532952339, 0.09527869648997933, -0.25]

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 15]

leg_num = 0

plt.figure(figsize=(10,15),
            facecolor='white')
# plot for x
p_ref_theor = [x[0] for x in p_ref_hist]
p_ref_pract = [x[0] for x in ef_pos_pract]

plt.subplot(3, 1, 1)
plt.plot(t_axis, p_ref_pract, color = 'r', label='Simulated value') # pract
plt.plot(t_axis, p_ref_theor[1:], color = 'g', label='Calculated value') # theor
plt.legend()
plt.grid()
plt.ylabel("x (m)")


# plot for y
p_ref_theor = [x[1] for x in p_ref_hist]
p_ref_pract = [x[1] for x in ef_pos_pract]

plt.subplot(3, 1, 2)
plt.plot(t_axis, p_ref_pract, color = 'r', label='Simulated value') # pract
plt.plot(t_axis, p_ref_theor[1:], color = 'g', label='Calculated value') # theor
plt.legend()
plt.grid()
plt.ylabel("y (m)")

# plot for z
p_ref_theor = [x[2] for x in p_ref_hist]
p_ref_pract = [x[2]-0.5 for x in ef_pos_pract]

plt.subplot(3, 1, 3)
plt.plot(t_axis, p_ref_pract, color = 'r', label='Simulated value') # pract
plt.plot(t_axis, p_ref_theor[1:], color = 'g', label='Calculated value') # theor
plt.legend()
plt.grid()
plt.ylabel("z (m)")
plt.xlabel("time (sec)")


plt.suptitle("Inverse kinematics check for leg #{0}".format(leg_num+1))
plt.show()